# Email orgainzer

needs:
-I have emails across many accounts 
-I want a single view of all pending emails 
-I want to be able to pull those emails 
-I want to see the history of those emails and understand which fall into the same threads so they can be clustered and triaged at the thread level

## 1
Prompt:
write a function 

email_pull(acct_dict, date_range)

pulls email from a given dictionary of accounts argument acct_dict
{account_nm:acct_pswd, account_nm:acct_pswd,account_nm:acct_pswd ...} from a given date date range argument "date_range": start_date:end_date. Store the body of the email in a spreadsheet with headers acct_dict, date_range, body, metadata, thread_nm and save it to a dataframe. return the dataframe.

## 2
write a function 

email_pull(acct_dict, date_range)

take the output datframe from 


## 3
generate a dash visulization for the resulting graph


In [ ]:
import imaplib
import email
import pandas as pd
from datetime import datetime, timedelta
from email.header import decode_header, make_header

def email_pull(acct_dict: dict, date_range: str) -> pd.DataFrame:
    """
    acct_dict: { account_email: password, ... }
    date_range: "YYYY-MM-DD:YYYY-MM-DD"
    returns DataFrame with columns:
      account, date_range, body, metadata, thread_nm
    """
    records = []
    start_str, end_str = date_range.split(":")
    start_dt = datetime.strptime(start_str, "%Y-%m-%d")
    end_dt = datetime.strptime(end_str,   "%Y-%m-%d") + timedelta(days=1)

    # IMAP wants dates like '01-Jan-2025'
    def imap_date(dt: datetime) -> str:
        return dt.strftime("%d-%b-%Y")

    since = imap_date(start_dt)
    before = imap_date(end_dt)

 # where to dump attachments
    attach_base = os.path.join(os.getcwd(), "email_attachments")
    
  
    for acct, pwd in acct_dict.items():
        try:
            m = imaplib.IMAP4_SSL("imap.gmail.com", 993)   # adjust host/port per provider
            m.login(acct, pwd)
            m.select("INBOX")
            typ, data = m.search(
                None,
                f'(SINCE "{since}" BEFORE "{before}")'
            )
            for num in data[0].split():
                typ, msg_data = m.fetch(num, "(RFC822)")
                raw = msg_data[0][1]
                msg = email.message_from_bytes(raw)

                # decode thread_nm from Subject
                subj = str(make_header(decode_header(msg.get("Subject", ""))))
                thread_nm = subj

                # extract metadata
                metadata = {
                    "from": msg.get("From"),
                    "to":   msg.get("To"),
                    "date": msg.get("Date"),
                    "subject": subj
                }

                # extract body (plain text first, else HTML)
                body = ""
                if msg.is_multipart():
                    for part in msg.walk():
                        ctype = part.get_content_type()
                        disp  = part.get("Content-Disposition", "")
                        if ctype == "text/plain" and "attachment" not in disp:
                            body = part.get_payload(decode=True).decode(errors="ignore")
                            break
                    else:
                        # fallback to first text/html
                        for part in msg.walk():
                            if part.get_content_type() == "text/html":
                                body = part.get_payload(decode=True).decode(errors="ignore")
                                break
                else:
                    body = msg.get_payload(decode=True).decode(errors="ignore")

                records.append({
                    "account": acct,
                    "date_range": date_range,
                    "body": body,
                    "metadata": metadata,
                    "thread_nm": thread_nm, 
                    "attachments": attachments or None
                })

            m.logout()
        except Exception as e:
            print(f"Error pulling {acct}: {e}")

    df = pd.DataFrame(records, columns=[
        "account", "date_range", "body", "metadata", "thread_nm", "attachments"
    ])
    return df

##### Suggested to pull Attachments #######
    # attach_base = os.path.join(os.getcwd(), "email_attachments")
    
    # for acct, pwd in acct_dict.items():
    #     # … login/search …
    #     for num in data[0].split():
    #         # fetch message
    #         msg = email.message_from_bytes(msg_data[0][1])

    #         # make sure per‐account attach dir exists
    #         acct_dir = os.path.join(attach_base, acct)
    #         os.makedirs(acct_dir, exist_ok=True)

    #         # init
    #         body = ""
    #         attachments = []

    #         if msg.is_multipart():
    #             for part in msg.walk():
    #                 ctype = part.get_content_type()
    #                 disp  = part.get("Content-Disposition", "")
                    
    #                 # attachments
    #                 if disp and disp.strip().startswith("attachment"):
    #                     fname = str(make_header(decode_header(part.get_filename())))
    #                     path  = os.path.join(acct_dir, fname)
    #                     with open(path, "wb") as fp:
    #                         fp.write(part.get_payload(decode=True))
    #                     attachments.append(path)
                    
    #                 # first plain‐text part as body
    #                 elif ctype == "text/plain" and "attachment" not in disp:
    #                     body = part.get_payload(decode=True).decode(errors="ignore")

    #             # fallback to HTML if no plain‐text found
    #             if not body:
    #                 for part in msg.walk():
    #                     if part.get_content_type() == "text/html":
    #                         body = part.get_payload(decode=True).decode(errors="ignore")
    #                         break

    #         else:
    #             # not multipart → whole payload is body
    #             body = msg.get_payload(decode=True).decode(errors="ignore")

    #         # metadata/thread as before…
    #         subj     = str(make_header(decode_header(msg.get("Subject",""))))
    #         thread_nm= subj
    #         metadata = {
    #             "from":    msg.get("From"),
    #             "to":      msg.get("To"),
    #             "date":    msg.get("Date"),
    #             "subject": subj
    #         }

    #         records.append({
    #             "account":     acct,
    #             "date_range":  date_range,
    #             "body":        body,
    #             "metadata":    metadata,
    #             "thread_nm":   thread_nm,
    #             "attachments": attachments or None
    #         })

    #     m.logout()

    # df = pd.DataFrame(records, 
    #          columns=["account","date_range","body","metadata","thread_nm","attachments"])
    # return df


In [ ]:
# ### EMAIL CREDENTIALS SETTUP
# if __name__ == "__main__":
#     base64mode=False

#     ############### API credential loading from local folder  ########
#     #### secrets_ls_file1: is the financial tracker service acct
#     #### secrets_ls_file2: are the kanban boards individual APIs

#     verbose=False
#     #method=input("which connection method should I use (local or colab_auth)?") 
#     method = 'L'

#     g_conect, api_dict=acd.load_method(method, secrets_ls=
#             ['/Users/gunnarkleemann/.ssh/financetrack-acd-aa2d12d1f346.json',
#             '/Users/gunnarkleemann/.ssh/acd-api.json'])

#     #/Users/gunnarkleemann/.ssh/acd-internal-analytics-375db6d96d79.json

#     api_token_edu_board=api_dict['api_token_edu_board']
#     api_token_contr_board=api_dict['api_token_contract_board']

#     folder_path = "/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_financial/Invoices/test_inv/FromSvcAcct_Incoming"  # Folder with PDF invoices
#     lama_endpoint = "http://localhost:11434/api/process"#/process"  # LaMA 3.1 API endpoint
#     #"/Users/gunnarkleemann/.ssh/acd-internal-analytics-375db6d96d79.json"
#     credentials_file = '/Users/gunnarkleemann/.ssh/financetrack-acd-aa2d12d1f346.json' # Google service account key
#     share_with_emails = ["gunnar@austincapitaldata.com", "gunnarkl@gmail.com"]  # Emails to share the sheets with
#     sender_email = "service@austincapitaldata.com"  # Sender email for notifications
#     sender_password = api_dict['service_acct_email_psswd']  # Sender email password (use app-specific passwords if needed)- this can't be your email login, go to 'Google App Passwords'


# securely prompt for your password
from getpass import getpass

# replace with your real email address
acct_dict = {
    "service@austincapitaldata.com": getpass("Application Passwd service@gmail.com")
}

# set the date range you want to pull
date_range = "2024-12-01:2025-12-01"

# call your function
df = email_pull(acct_dict, date_range)
df.head()

,account,date_range,body,metadata,thread_nm
0,service@austincapitaldata.com,2024-12-01:2025-12-01,Attached: Invoice for October.pdf\r\nSent usin...,"{'from': '""Kirti kamerkar (via Google Docs)"" <...",Invoice for November
1,service@austincapitaldata.com,2024-12-01:2025-12-01,Sharing with you in case you can suggest some ...,"{'from': 'Lynn Gant <lgant@accelebrate.com>', ...",FW: bio update favor
2,service@austincapitaldata.com,2024-12-01:2025-12-01,Yes – sure does!thx\r\n\r\nLynn Gant\r\n\r\nPh...,"{'from': 'Lynn Gant <lgant@accelebrate.com>', ...",RE: FW: bio update favor
3,service@austincapitaldata.com,2024-12-01:2025-12-01,Hi Gunnar!\r\n\r\nI am reaching out for the in...,{'from': 'Melissa Greenhalgh <melissa.greenhal...,Request for Invoice/receipt copies - Deloitte ...
4,service@austincapitaldata.com,2024-12-01:2025-12-01,Kevin - the Gen AI content for the Deloitte Co...,"{'from': 'Lynn Gant <lgant@accelebrate.com>', ...",Feb 24 - can you help this day ? need you Feb ...
